# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [11]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [25]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [26]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [28]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [15]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [16]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [17]:
gamma = 1
sigma = .005

In [18]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [19]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [20]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Considere un swap a 1Y con cupones semestrales de 2.0% (lineal). Si la tasa
cupón cero a 0.5Y es 1.9% (exp), calcule la tasa cero a 1Y.


Pregunta 2:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a -.7028 .


Pregunta 3:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,049.00 (365 días después).


Pregunta 4:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 5:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 582 y 754 días.


Pregunta 6:

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.


Pregunta 7:

Explique la diferencia ent

## Respuestas

### Pregunta 1

### Pregunta 2

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a -.7028 .

In [50]:
from typing import List, Tuple


In [52]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}
gamma = 1
sigma = .05
r_ = .01
r0 = .01 # tasa corta a día de hoy

def vasicek_path(r0: float,
                 gamma: float, r_: float,
                 sigma: float,
                 num_dias: int = 263,
                 dias_agno: int = 264,
                 pasos_dia: int = 1) -> List[Tuple[float, float]]:
    """
    Retorna un camino de simulación del modelo de Vasicek.
    
    params:
    
    - r0: tasa inicial (t = 0) de la simulación
    - Parámetros del modelo:
      - gamma: velocidad de reversión
      - r_: tasa de largo plazo
      - sigma: volatilidad
    - num_dias: número de días en la trayectoria, incluyendo el instante t = 0
    - dias_agno: número de días hábiles por año
    - pasos_dia: número de pasos de simulación en 1 día
    
    return:
    
    - Un `list` donde cada elemento es una `tuple` con los valores del tiempo y la tasa simulada.
    """
    dt = 1 / (dias_agno * pasos_dia)
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0),]
    gamma_dt = gamma * dt
    
    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * np.random.normal() # Discretización de Euler
        result.append((i * dt, r))
    return result


sim = vasicek_path(r0, gamma, r_, sigma, num_dias=1000)
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])
fig = px.line(
    df_sim,
    x='t',
    y='tasa',
    title=f'Simulación Modelo Vasicek.'
)
fig.show()


### Pregunta 3

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,049.00 (365 días después)

$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$


In [56]:
icp0=10000
icp365=10049
d=365
TNA=round((icp0/icp365-1)*360/d,4)
print(TNA)

-0.0048


### Pregunta 4

### Pregunta 5

In [30]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear')
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [35]:
t1=582
t2=754

r1=curva(t1)
r2=curva(t2)
df1=math.exp(-r1*t1 /365)
df2=math.exp(-r2*t2 /365)
fwd=(df1/df2 - 1) * 360 /(t2-t1)
print(fwd)

0.00041163793062275157


### Pregunta 6

Verdadero. Para el cálculo de opciones, call y put. Las formulas empledas en ambos modelos son las mismas:

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$

$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T_B\right)}{KZ\left(r_0,0,T_O\right)}\right)+\frac{1}{2}S_Z\left(T_O\right)^2}{S_Z\left(T_O\right)}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T_O\right)
\end{equation}
$$


$$
\begin{equation}
S_Z\left(T_O\right)=B\left(T_O,T_B\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp\left(-2\gamma^* T_O\right)\right)}
\end{equation} 
$$

Y para una put:

$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

La diferencia entre estos dos modelos radica en que el modelo de HW incorpora un parámetro theta que es estocástico, varía en el tiempo. Este parámetro aparece de forma implícita dentro del cálculo del precio, lo que permite calibrarlo y por ende ajustar perfectamente la curva del modelo al precio de mercado.



### Pregunta 7

La primera diferencia, es que al hacer el cambio de medida, cambia la probabilidad de acceder o llegar a los distintos caminos es decir, cambia la formula para el drift, a pesar de que la volatilidad es la misma.

Drift en la medida ajustada por riesgo:

$$
\begin{equation}
\left(\theta_t-\gamma^* r_t\right)
\end{equation}
$$


Drift en la medida forward:

$$
\theta_t-B\left(t,T\right)\sigma^2-\gamma^* r_t
$$

Finalmente, la gran diferencia en terminos de procedimiento, radica en la tasa. 

En la medida libre de riesgo, se debe discretizar el path con el factor de descuento para cada instante t. Es por esto, que calculamos todos los "df", luego los multiplicamos entre si,  y finalmente multiplicamos por el nocional (=1 en bonos cero cupon) para traer a valor presente.

Mientras que en la medida forward se descuenta utilizando la misma tasa t en que vence el contrato y luego se multiplica por el valor esperado del payoff en la fecha de vencimiento.

